In [1]:
def weight(n):
    if n.parent().is_prime_field():
        p = n.parent().order()
        val = int(n)

        # Consider both positive and negative
        pos = val
        neg = abs(val - p)

        def hamming(n):
            return bin(abs(n)).count('1')

        return min(hamming(pos), hamming(neg))
    else:
        return sum(weight(c) for c in n.list())

In [2]:
def balanced_hex(n):
    if n.parent().is_prime_field():
        p = n.parent().order()
        val = int(n)

        # Consider both positive and negative
        pos = val
        neg = abs(val - p)

        def hamming(n):
            return bin(abs(n)).count('1')

        if hamming(pos) <= hamming(neg):
            return f" 0x{pos:08x}"
        else:
            return f"-0x{neg:08x}"
    else:
        return ' '.join([balanced_hex(c) for c in n.list()])


In [3]:
p = 2^31 - 1
F = GF(p)
assert F.is_field()

R.<X> = F[]
F2.<i> = F.extension(X^2 + 1)
assert F2.is_field()

R.<X> = F2[]
F6.<j> = F2.extension(X^3 - 5)
assert F6.is_field()


In [4]:
# Multiplicative orders:
print(factor(F.order()-1))
print(factor(F2.order()-1))
print(factor(F6.order()-1))

2 * 3^2 * 7 * 11 * 31 * 151 * 331
2^32 * 3^2 * 7 * 11 * 31 * 151 * 331
2^32 * 3^3 * 7 * 11 * 13 * 31 * 43^2 * 79 * 151 * 331 * 1381 * 529510939 * 1758566101 * 2903110321


In [8]:
F2_generator = F2.multiplicative_generator()
F2_order = F2_generator.multiplicative_order()

def F2_omega(n):
    assert F2_order % n == 0
    return F2_generator^(F2_order / n)

In [11]:
F2_omega(2^32)

1117296306*i + 1166849849

In [20]:
best = 100
root = F2_omega(2^32)
root2 = root * root
power = root
for i in range(2^22):
    if weight(power) < best:
        best = weight(power)
        print(balanced_hex(power), best)
    power *= root2

-0x3a7348c6  0x429896b2 28
-0x546ea819 -0x22bd0402 24
-0x40a78b85  0x60348b00 22
 0x54508594  0x104800eb 20
 0x1225542e  0x00203624 19
 0x02862052 -0x08406151 16
-0x46222494  0x48400808 15
 0x1c001880 -0x00206d24 14
 0x080840a1 -0x00038303 13
-0x20548084 -0x04140c00 12
 0x00164250 -0x22140000 11


In [21]:
# Up to omage_8 there is nice structure requiring only bitshifts and signs
def _():
    N = 8
    omega = F2_omega(N)
    for n in range(0, N):
        val = omega^n
        print(balanced_hex(val))
_()

 0x00000001  0x00000000
 0x00008000 -0x00008000
 0x00000000 -0x00000001
-0x00008000 -0x00008000
-0x00000001  0x00000000
-0x00008000  0x00008000
 0x00000000  0x00000001
 0x00008000  0x00008000


In [22]:
0x00008000

32768

In [23]:
2^15

32768

In [16]:
balanced_hex(F2_omega(2^32)^4)

' 0x1e6299e2 -0x5049d5cc'

In [175]:
basis = [1, i, j, i * j, j^2, i * j^2]

In [176]:
def to_coeff(n):
    if n.parent() == F:
        return [n]
    if n.parent() == F2:
        return n.list()
    if n.parent() == F6:
        return sum([to_coeff(m) for m in n.list()], [])
    assert False    

In [177]:
def from_coeff(c):
    return sum([c * b for (c, b) in zip(c, basis)])

In [182]:
for _ in range(1000):
    n = F6.random_element()
    assert n == from_coeff(to_coeff(n))

In [183]:
def embed(n):
    if n.parent() == F:
        return n
    if n.parent() == F2:
        return n.conjugate()
    if n.parent() == F6:
        c = [embed(c) for c in n.list()]
        return c[0] + (c[2] * j + c[1] * j^2) / 5
    assert False

In [184]:
for _ in range(1000):
    va = [F.random_element() for _ in range(6)]
    vb = [F.random_element() for _ in range(6)]
    vc = sum([a * b for (a,b) in zip(va, vb)])

    a = from_coeff(va)
    b = embed(from_coeff(vb))
    c = a * b

    assert to_coeff(c)[0] == vc

In [191]:
F6.field()

AttributeError: 'PolynomialQuotientRing_field_with_category' object has no attribute 'field'

In [186]:
F2.multiplicative_generator()

i + 12

In [35]:
F2(i + 12).multiplicative_order()

4611686014132420608

In [36]:
factor(F2(i + 12).multiplicative_order())

2^32 * 3^2 * 7 * 11 * 31 * 151 * 331

In [37]:
F2g = F2.multiplicative_generator()
F2order = F2g.multiplicative_order()
print(F2g)
print(F2order ,' = ', factor(F2order))

i + 12
4611686014132420608  =  2^32 * 3^2 * 7 * 11 * 31 * 151 * 331


In [39]:
F2_omega(2^32)

1117296306*i + 1166849849

In [40]:
# Up to omage_8 there is nice structure requiring only bitshifts and signs
def _():
    N = 8
    omega = F2_omega(N)
    for n in range(0, N):
        val = omega^n
        print(balanced_hex(val))
_()

 0x00000001  0x00000000
 0x00008000 -0x00008000
 0x00000000 -0x00000001
-0x00008000 -0x00008000
-0x00000001  0x00000000
-0x00008000  0x00008000
 0x00000000  0x00000001
 0x00008000  0x00008000


In [41]:
balanced_hex(F2(2147483646))

'-0x00000001  0x00000000'

In [42]:
R2.<y> = F2[]

In [45]:
P2 = y^3 - i - 2

In [46]:
P2.is_irreducible()

True

In [47]:
(i + 2)^(-1)

1288490188*i + 1717986918

In [253]:
F6.<j> = F2.extension(P2)

In [254]:
F6

Univariate Quotient Polynomial Ring in j over Finite Field in i of size 2147483647^2 with modulus j^3 + 2147483642

In [255]:
j^3

5

In [221]:
frob = F6.frobenius_endomorphism()
for e in range(7):
    print(frob(j^e))

1
1513477735*j
634005911*j^2
5
1124937734*j
1022545908*j^2
25


In [222]:
F6(1513477735)

1513477735

In [95]:
# Low hamming weight values
simple_values = [F(0), F(1), F(-1), F(5), F(-5), F(3), F(-3)] + [F(2^n) for n in range(1, 31)] + [F(-2^n) for n in range(1, 32)]

In [96]:
simple_values = simple_values[:10]
simple_values

[0, 1, 2147483646, 5, 2147483642, 3, 2147483644, 2, 4, 8]

In [97]:
best = 3
for a in simple_values:
    for b in simple_values:
        for c in simple_values:
            for d in simple_values:
                for e in simple_values:
                    for f in simple_values:
                        A = a + b * i
                        B = c + d * i
                        C = e + f * i
                        P2 = A + B * y + C * y^2 + y^3
                        if not P2.is_irreducible():
                            continue
                        w = weight(P2)
                        if w <= best:
                            best = w
                            print(w, ' => ', P2)

3  =>  y^3 + 8*i*y^2 + i
3  =>  y^3 + 8*y^2 + i
3  =>  y^3 + 8*i*y + i
3  =>  y^3 + 8*y + i
3  =>  y^3 + 8*y^2 + 2147483646*i
3  =>  y^3 + 8*i*y + 2147483646*i
3  =>  y^3 + 8*y + 2147483646*i
3  =>  y^3 + 5*i
3  =>  y^3 + 2147483642*i
3  =>  y^3 + 2*i*y^2 + 2*i
3  =>  y^3 + 4*i*y^2 + 2*i
3  =>  y^3 + 8*y^2 + 2*i
3  =>  y^3 + i*y + 2*i
3  =>  y^3 + 2147483646*i*y + 2*i
3  =>  y^3 + 8*i*y + 2*i
3  =>  y^3 + 4*y + 2*i
3  =>  y^3 + 4*i*y^2 + 4*i
3  =>  y^3 + y^2 + 4*i
3  =>  y^3 + 2147483646*y^2 + 4*i
3  =>  y^3 + 8*y^2 + 4*i
3  =>  y^3 + i*y + 4*i
3  =>  y^3 + 2147483646*i*y + 4*i
3  =>  y^3 + 2147483646*y + 4*i
3  =>  y^3 + 4*y + 4*i
3  =>  y^3 + 8*y + 4*i
3  =>  y^3 + i*y + 8*i
3  =>  y^3 + 2147483646*i*y + 8*i
3  =>  y^3 + 8*i*y^2 + 1
3  =>  y^3 + 8*i*y + 1
3  =>  y^3 + 2*i + 1
3  =>  y^3 + 4*i + 1
3  =>  y^3 + 8*i*y^2 + 2147483646
3  =>  y^3 + 8*y^2 + 2147483646
3  =>  y^3 + 8*i*y + 2147483646
3  =>  y^3 + 2*i + 2147483646
3  =>  y^3 + 4*i + 2147483646
3  =>  y^3 + 5


KeyboardInterrupt: 

In [135]:
P2 = y^3 - 5

NameError: name 'y' is not defined

In [264]:
P2.is_irreducible()

True

In [265]:
F6.<j> = F2.extension(P2)

In [266]:
F6

Univariate Quotient Polynomial Ring in j over Finite Field in i of size 2147483647^2 with modulus j^3 + 2147483642

In [259]:
F6.is_field()

True

In [260]:
M = FiniteRankFreeModule(F2, 3, name='M') ; M

3-dimensional vector space M over the Finite Field in i of size 2147483647^2

In [261]:
e = M.basis('e') ; e

Basis (e_0,e_1,e_2) on the 3-dimensional vector space M over the Finite Field in i of size 2147483647^2

In [262]:
t = M.tensor((3,0), name='t') ; t

Type-(3,0) tensor t on the 3-dimensional vector space M over the Finite Field in i of size 2147483647^2

In [272]:
t[0,0,0] = -3

In [273]:
variables = []
for a in range(3):
    variables += [f'C_{a}0']
    for b in range(3):
        p = j^a * j^b
        variables += [f'A_{a}{b}', f'B_{a}{b}']
        for c in range(3):
            t[a,b,c] = p.list()[c]

In [274]:
R = PolynomialRing(F2, variables)
variables = R.gens_dict()
R

Multivariate Polynomial Ring in C_00, A_00, B_00, A_01, B_01, A_02, B_02, C_10, A_10, B_10, A_11, B_11, A_12, B_12, C_20, A_20, B_20, A_21, B_21, A_22, B_22 over Finite Field in i of size 2147483647^2

In [329]:
eqs = []
for d in range(3):
    for e in range(3):
        lhs = 0
        for a in range(3):
            for b in range(3):
                for c in range(3):
                    A = 1 if a == d else 0 # variables[f'A_{a}{d}']
                    B = variables[f'B_{b}{e}']
                    C = 1 if c < 1 else 0 # variables[f'C_{c}0']
                    lhs += t[a,b,c] * A * B * C
        rhs = 1 if d == e else 0
        eqs += [lhs - rhs]
eqs

[B_00 - 1, B_01, B_02, 5*B_20, 5*B_21 - 1, 5*B_22, 5*B_10, 5*B_11, 5*B_12 - 1]

In [330]:
I = Ideal(eqs)
I

Ideal (B_00 - 1, B_01, B_02, 5*B_20, 5*B_21 - 1, 5*B_22, 5*B_10, 5*B_11, 5*B_12 - 1) of Multivariate Polynomial Ring in C_00, A_00, B_00, A_01, B_01, A_02, B_02, C_10, A_10, B_10, A_11, B_11, A_12, B_12, C_20, A_20, B_20, A_21, B_21, A_22, B_22 over Finite Field in i of size 2147483647^2

In [331]:
G = I.groebner_basis(); G

[B_00 - 1,
 B_01,
 B_02,
 B_10,
 B_11,
 B_12 - 858993459,
 B_20,
 B_21 - 858993459,
 B_22]

In [333]:
1/F2(858993459)

5

In [325]:
F2.characteristic()

2147483647

In [122]:
F6.<x> = GF(p^6)

In [124]:
F6(x^3).frobenius()

739530791*x^5 + 1223070150*x^4 + 540426349*x^3 + 682643827*x^2 + 1962600948*x + 1450618083

In [134]:
F6.modulus()

x^6 + 5*x^5 + 2147483645*x^4 + 2147483622*x^3 + 2147483638*x^2 + 20*x + 5

In [127]:
for e in range(7):
    print(balanced_hex(F6(x^e).frobenius()))

 0x00000001  0x00000000  0x00000000  0x00000000  0x00000000  0x00000000
-0x305160a5 -0x48e691ce  0x348e6921 -0x210f421d -0x6c80d902  0x239a4734
-0x247348e3  0x036406c4 -0x2036406e  0x312a6256  0x10f421e9 -0x0d901b20
-0x2989531c -0x0b05160b  0x28b05173  0x2036406d  0x48e691c6  0x2c145827
-0x22c14578 -0x4d239a66  0x1cd2399c -0x1e843cf2 -0x01b20358  0x348e691e
 0x087a10da -0x12a62512  0x312a62a5 -0x0e691d10  0x22c14543  0x4a989526
-0x0000000e -0x00000180 -0x0000003c  0x0000016a  0x000000cf  0x0000001d
